In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Look Closer At Data

In [ ]:
data = pd.read_csv('/kaggle/input/the-world-of-pokemons/pokemons dataset.csv',encoding='latin-1')

In [ ]:
data.head()

# Visualize data and find any clue

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
li = ['Attack','Defense','HP','Sp.Attack','Sp.Defense','Speed']
plt.figure(figsize=(10,8))
for i in enumerate(li):
  plt.subplot(2,3,i[0]+1)
  sns.scatterplot(data[i[1]],data['Total'])
  

* *We can see here there are some common outliers*  *

# find more detailed look using boxplot to find outliers

In [ ]:
li = ['Attack','Defense','HP','Sp.Attack','Sp.Defense','Speed','Total']
for i in enumerate(li):
  plt.subplot(3,3,i[0]+1)
  sns.boxplot(data[i[1]])

# Remove outlier and then normalize data

In [ ]:
def remove(data):
  Q1 = np.percentile(data, 25, interpolation = 'midpoint')
  
# Third quartile (Q3)
  Q3 = np.percentile(data, 75, interpolation = 'midpoint')
  iqr = Q3 - Q1
  percentile25 = data.quantile(0.25)
  percentile75 = data.quantile(0.75)

  upper_limit = percentile75 + 1.5 * iqr
  lower_limit = percentile25 - 1.5 * iqr
  data = np.where(
    data > upper_limit,upper_limit,
    np.where(
        data < lower_limit,
        lower_limit,
        data
    )
  )
  return data

#train[train['discount_percent'] > upper_limit]
#train[train['discount_percent'] < lower_limit]



# but first of do convert string data to int or category data

In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [ ]:
data['Name2'] = np.where(data['Name2'].notnull(),1,0)
data['Secondary type'] = np.where(data['Secondary type'].isnull(),'0',data['Secondary type'])
data['Primary Type']  = LabelEncoder().fit_transform(data['Primary Type'])
data['Secondary type']  = LabelEncoder().fit_transform(data['Secondary type'])

In [ ]:
print(data.shape)
data =data.drop_duplicates(keep='first')
print(data.shape)




In [ ]:
plt.figure(figsize=(10,8),dpi=100)
li = ['Name2','Secondary type','Primary Type','Attack','Defense','HP','Sp.Attack','Sp.Defense','Speed','Total']
for i in enumerate(li):
  plt.subplot(5,3,i[0]+1)
  sns.boxplot(data[i[1]])

In [ ]:
data['Name2'].unique()

In [ ]:
data.head()

In [ ]:
for i in data.columns[2:]:
  data[i]=remove(data[i])
  data[i] = StandardScaler().fit_transform(data[[i]])

In [ ]:
li = ['Attack','Defense','HP','Sp.Attack','Sp.Defense','Speed','Total']
for i in enumerate(li):
  plt.subplot(3,3,i[0]+1)
  sns.boxplot(data[i[1]])

In [ ]:
data['Name'].unique()

** As there have so many names I avoid this column though I can use maximum occured name  **

In [ ]:
sns.heatmap(data.corr(),fmt = '.1f',annot=True,mask=np.triu(data.corr()))

# Split feature and label


In [ ]:
feature =data.drop(['Total','Name'],axis=1)
label = data['Total']

# Split train and test data

In [ ]:
from sklearn.model_selection import train_test_split
trainF,testF,trainL,testL = train_test_split(feature,label,random_state=42,test_size=.3)

# Choose model , in that case I use *CATBOOSTREGRESSOR*

In [ ]:
import catboost as cb
model = cb.CatBoostRegressor(random_state=42)

In [ ]:
model.fit(trainF,trainL)

In [ ]:
model.score(testF,testL)

# hurraay 99%

# Lets Visualization for satisfaction

In [ ]:
pred = model.predict(testF)
pred = np.array(pred)
testL = np.array(testL)
plt.plot(pred[:50])
plt.plot(testL[:50])
plt.legend(['pred','actual'])